# Course Project - Convex Optimization for Signal Processing and Communications

- Student: Lucas von Ancken Garcia
- Matrikelnummer: 2576600 

## Problem 1

Given a single-user (point-to-point) MIMO communication channel with the channel matrix $H$ and zero mean
colored noise $n$ at the receiver antennas, the received symbol at the receiver is $y = H^H \cdot x + n$, where $x$ is the
transmitted symbol with zero mean. We want to **design the optimal transmit covariance matrix** that **maximizes the transmission rate** of the MIMO system. We assume that some of the **antennas have power constraints** that
restrict the transmitted power. Furthermore, **a sum power constraint applies**.

Generate required data to implement the problem formulation in CVX. Average the simulation results with Monte Carlo iterations.

In [2]:
import numpy as np
import cvxpy as cvx
from scipy import signal


np.random.seed(1)
np.set_printoptions(precision=3, suppress=True)

In [3]:
# Generation of the complex Channel Matrix H according to Rayleigh Fading Model
def generate_rayleigh_channel_matrix(m, n):
    real_part = np.random.normal(0, np.sqrt(0.5), (m,n))
    imag_part = np.random.normal(0, np.sqrt(0.5), (m,n))

    return real_part + 1j*imag_part

def generate_white_noise(n):
    real_part = np.random.normal(0, np.sqrt(0.5), n)
    imag_part = np.random.normal(0, np.sqrt(0.5), n)

    return real_part + 1j*imag_part

def generate_random_channel_vector(n, mean, std):
    real_part = np.random.normal(mean, std, n)
    imag_part = np.random.normal(mean, std, n)
    
    return (real_part + 1j*imag_part).reshape(n,1)

In [48]:
num_tx = 6  # Number of transmit antennas
num_rx = 4  # Number of receive antennas

power_noise = 3
total_power = 60

power_limit_per_tx = [9, 20, 10, 8, 12, 16]

H = cvx.Parameter((num_tx, num_rx), complex=True)
Rn_inv = cvx.Parameter((num_rx, num_rx))

P = cvx.Variable((num_tx, num_tx), symmetric=True)

objective = cvx.Maximize(cvx.log_det(np.identity(num_rx) + Rn_inv @ cvx.conj(H.T) @ P @ H))

# constraints = [P >> 0,  P == P.T, cvx.trace(P) <= total_power]
constraints = [P >> 0, cvx.trace(P) <= total_power]

for i in range(num_tx):
    constraints.append(P[i, i] <= power_limit_per_tx[i])

prob = cvx.Problem(objective, constraints)

H.value = generate_rayleigh_channel_matrix(num_tx, num_rx)
Rn_inv.value = np.identity(num_rx)*(1/power_noise)

prob.solve(verbose=True)

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 17 09:08:31 AM: Your problem has 36 variables, 43 constraints, and 40 parameters.
(CVXPY) Jun 17 09:08:31 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 17 09:08:31 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 17 09:08:31 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 17 09:08:31 AM: Compiling problem (target solver=SCS).
(CVXPY) Jun 17 09:08:31 AM: Reduction chain: EvalParams -> Complex2Real -> FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStu

c:\Users\User\Documents\Selbststudium\TUDa_MachineLearning-DS-Exercises\Convex Optimization\Project\.venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:235: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)


11.198219938104193

In [49]:
print("CONSTANTS")
print("")
print("Number of Transmitters: ", num_tx)
print("Number of Receivers: ", num_rx)
print("Power Limits per TX: {} (sum = {})".format(power_limit_per_tx, sum(power_limit_per_tx)))
print("Total Power Limit", total_power)
print("")
print("")

print("INPUT PARAMETERS")
print("")
print("Power noise: ", power_noise)
print("Channel Matrix  H = ")
print(H.value)
print("")
print("")

print("RESULTS")
print("")
print("Status:", prob.status)
print("Optimal value: ", objective.value)
print("Num of Constraints: ", len(constraints))
print("Eigenvalues of optimal P = ", np.linalg.eigvalsh(P.value))
print("")
print("--------------------")
print("P* = ")
print(P.value)
print("Actual Total Power of the Optimal Solution: {}".format(P.value.trace()))
print("--------------------")
print("Lagrange Multipliers")
print("For P >= 0: ")
print(constraints[0].dual_value)
print("Eigenvalues of Langrange Multiplier of P >=0 : ", np.linalg.eigvalsh(constraints[0].dual_value))
print("For total power:")
print(constraints[1].dual_value)
print("For the limit power of each TX:")
print([x.dual_value for x in constraints[2:]])

CONSTANTS

Number of Transmitters:  6
Number of Receivers:  4
Power Limits per TX: [9, 20, 10, 8, 12, 16] (sum = 75)
Total Power Limit 60


INPUT PARAMETERS

Power noise:  3
Channel Matrix  H = 
[[ 1.149+0.637j -0.433-0.483j -0.373-0.087j -0.759-0.662j]
 [ 0.612-0.189j -1.627+0.375j  1.234-0.489j -0.538-0.281j]
 [ 0.226-0.486j -0.176-0.598j  1.034-0.475j -1.457-0.009j]
 [-0.228-0.79j  -0.272+0.166j  0.802+1.174j -0.778+0.525j]
 [-0.122-0.136j -0.621-0.628j  0.03 -0.528j  0.412+1.197j]
 [-0.778+0.036j  0.809-0.45j   0.638+0.135j  0.355+1.485j]]


RESULTS

Status: optimal
Optimal value:  11.198219938104193
Num of Constraints:  8
Eigenvalues of optimal P =  [-0.    -0.    10.152 14.882 17.273 17.693]

--------------------
P* = 
[[ 9.     2.719 -0.819 -5.262  1.634 -1.67 ]
 [ 2.719 13.954  2.793  4.835  0.864 -2.416]
 [-0.819  2.793 10.    -1.57   3.774 -1.464]
 [-5.262  4.835 -1.57   8.    -0.569  2.531]
 [ 1.634  0.864  3.774 -0.569  8.516  7.201]
 [-1.67  -2.416 -1.464  2.531  7.201 10.

In [50]:
P_list = [P.value]
H_list = [H.value]
power_noise_list = [power_noise]
optimal_value_list = [prob.value]
for i in range(19):
    H.value = generate_rayleigh_channel_matrix(num_tx, num_rx)

    power_noise = np.random.uniform(low=0, high=5)
    Rn_inv.value = np.identity(num_rx)*(1/power_noise)

    prob.solve()

    H_list.append(H.value)
    power_noise_list.append(power_noise)

    P_list.append(P.value)
    optimal_value_list.append(prob.value)

H_avg = sum(H_list)/len(H_list)
power_noise_avg = sum(power_noise_list)/len(power_noise_list)

P_avg = sum(P_list)/len(P_list)
optimal_value_avg = sum(optimal_value_list)/len(optimal_value_list)

In [51]:
print("AVERAGE INPUT PARAMETERS")
print("")
print("Average power noise = ", power_noise_avg)
print("Average Channel Matrix H = ")
print(H_avg)
print("")
print("")

print("RESULTS")
print("")
print("Optimal Value Average: {}".format(optimal_value_avg))
print("Average P* = ")
print(P_avg)

AVERAGE INPUT PARAMETERS

Average power noise =  2.2629542302477113
Average Channel Matrix H = 
[[ 0.18 +0.222j  0.061+0.237j -0.058+0.152j -0.031-0.194j]
 [ 0.065+0.153j -0.22 +0.009j  0.12 +0.408j  0.131-0.008j]
 [ 0.317-0.014j  0.067-0.051j  0.053-0.021j -0.174+0.023j]
 [ 0.093-0.002j  0.064-0.261j -0.166+0.007j -0.075+0.167j]
 [ 0.004+0.031j -0.212+0.185j  0.02 +0.072j  0.057+0.283j]
 [-0.072-0.035j  0.151+0.115j -0.15 -0.14j   0.028+0.124j]]


RESULTS

Optimal Value Average: 13.882827520841483
Average P* = 
[[ 8.613  0.364 -0.854 -0.078 -0.569 -0.399]
 [ 0.364 12.143  0.717  0.549 -0.925 -0.787]
 [-0.854  0.717  8.696  0.17   0.223 -0.296]
 [-0.078  0.549  0.17   7.658  0.187  0.584]
 [-0.569 -0.925  0.223  0.187 11.005 -0.351]
 [-0.399 -0.787 -0.296  0.584 -0.351 11.885]]


## Problem 2
We assume further that a single-antenna co-channel user with channel vector hc is now present. The received symbol is $y_c = h^H_c \cdot x + n_c$, where $n_c$ is the zero mean colored noise at this user. We want that this user does not receive any interference in expectation, i.e., $E[∥y_c∥^2] = 0$.

Please modify the problem formulation to adopt the new constraint and solve it in CVX.

In [62]:
hc = generate_random_channel_vector(num_tx, 0, 5)


new_constraints = prob.constraints + [cvx.trace((hc @ hc.conjugate().transpose()) @ P) == 0]

new_prob = cvx.Problem(objective, new_constraints)
new_prob.solve(verbose=True, max_iters=int(2e6))

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 17 09:26:35 AM: Your problem has 36 variables, 44 constraints, and 40 parameters.
(CVXPY) Jun 17 09:26:35 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 17 09:26:35 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 17 09:26:35 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 17 09:26:35 AM: Compiling problem (target solver=SCS).
(CVXPY) Jun 17 09:26:35 AM: Reduction chain: EvalParams -> Complex2Real -> FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStu

c:\Users\User\Documents\Selbststudium\TUDa_MachineLearning-DS-Exercises\Convex Optimization\Project\.venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:235: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)


  2000| 9.55e-03  4.84e-06  1.19e-01 -1.60e+01  2.18e-03  1.68e-01 
  2250| 9.40e-03  4.80e-06  1.18e-01 -1.60e+01  2.18e-03  1.89e-01 
  2500| 9.26e-03  4.70e-06  1.18e-01 -1.60e+01  2.18e-03  2.09e-01 
  2750| 9.13e-03  4.64e-06  1.17e-01 -1.60e+01  2.18e-03  2.31e-01 
  3000| 9.00e-03  4.58e-06  1.16e-01 -1.60e+01  2.18e-03  2.51e-01 
  3250| 8.87e-03  4.53e-06  1.16e-01 -1.60e+01  2.18e-03  2.71e-01 
  3500| 6.42e-02  6.68e-04  8.76e-02 -1.59e+01  2.18e-03  2.90e-01 
  3750| 8.63e-03  4.44e-06  1.14e-01 -1.59e+01  2.18e-03  3.10e-01 
  4000| 8.52e-03  4.40e-06  1.14e-01 -1.59e+01  2.18e-03  3.30e-01 
  4250| 8.41e-03  4.36e-06  1.13e-01 -1.59e+01  2.18e-03  3.49e-01 
  4500| 8.30e-03  4.33e-06  1.13e-01 -1.59e+01  2.18e-03  3.69e-01 
  4750| 8.19e-03  4.29e-06  1.12e-01 -1.59e+01  2.18e-03  3.89e-01 
  5000| 8.09e-03  4.26e-06  1.11e-01 -1.59e+01  2.18e-03  4.08e-01 
  5250| 8.00e-03  4.23e-06  1.11e-01 -1.59e+01  2.18e-03  4.29e-01 
  5500| 7.90e-03  4.20e-06  1.10e-01 -1.59e+01  

15.768378744216061

In [6]:
hc = generate_random_channel_vector(6, 0, 5)
hc @ hc.transpose().conjugate()

array([[  2.643 +0.j   ,   3.711 +4.292j, -10.3   -7.675j,
         10.074 +8.066j,   2.341 +7.085j,   7.606 +3.124j],
       [  3.711 -4.292j,  12.179 +0.j   , -26.922 +5.952j,
         27.24  -5.035j,  14.792 +6.144j,  15.75  -7.966j],
       [-10.3   +7.675j, -26.922 -5.952j,  62.42  +0.j   ,
        -62.673 -2.182j, -29.695-20.81j , -38.709 +9.912j],
       [ 10.074 -8.066j,  27.24  +5.035j, -62.673 +2.182j,
         63.004 +0.j   ,  30.543+19.857j,  38.519-11.305j],
       [  2.341 -7.085j,  14.792 -6.144j, -29.695+20.81j ,
         30.543-19.857j,  21.065 +0.j   ,  15.11 -17.621j],
       [  7.606 -3.124j,  15.75  +7.966j, -38.709 -9.912j,
         38.519+11.305j,  15.11 +17.621j,  25.578 +0.j   ]])

In [63]:
interference =  np.trace(P.value @ (hc @ hc.transpose().conjugate()))

print("CONSTANTS")
print("")
print("Number of Transmitters: ", num_tx)
print("Number of Receivers: ", num_rx)
print("Power Limits per TX: {} (sum = {})".format(power_limit_per_tx, sum(power_limit_per_tx)))
print("Total Power Limit", total_power)
print("")
print("")

print("INPUT PARAMETERS")
print("")
print("Power noise: ", power_noise)
print("Channel Matrix  H = ")
print(H.value)
print("New user channel vector hc = ", hc.T)
print("")
print("")

print("RESULTS")
print("")
print("Status:", new_prob.status)
print("New optimal value: ", new_prob.value)
print("New num of Constraints: ", len(new_constraints))
print("--------------------")
print("new P* = ")
print(P.value)
print("Actual Total Power of the new Optimal Solution: {}".format(P.value.trace()))
print("Interference in new user: ", interference)
print("--------------------")
print("New Lagrange Multipliers")
print("For P >= 0: ")
print(new_constraints[0].dual_value)
print("For total power:")
print(new_constraints[1].dual_value)
print("For the limit power of each TX:")
print([x.dual_value for x in new_constraints[2:-1]])
print("For the zero interference in new user: ")
print(new_constraints[-1].dual_value)


CONSTANTS

Number of Transmitters:  6
Number of Receivers:  4
Power Limits per TX: [9, 20, 10, 8, 12, 16] (sum = 75)
Total Power Limit 60


INPUT PARAMETERS

Power noise:  0.3728703562573993
Channel Matrix  H = 
[[-0.279+0.661j -0.829+1.282j  0.696+0.099j -0.396-1.003j]
 [ 0.976-0.224j  0.426+0.453j -0.631+0.863j -0.114-0.802j]
 [-0.203-0.135j -0.616+0.165j  0.355+0.308j -0.338+0.644j]
 [ 1.154-0.671j  0.609-0.299j -0.622+0.713j -0.013+0.277j]
 [-0.16 +0.317j -1.106+0.796j  0.658+0.074j  0.672+0.373j]
 [ 0.654-0.222j -0.323-0.951j  0.756-0.916j -0.148+0.053j]]
New user channel vector hc =  [[-1.032+7.44j  -0.836+6.627j  2.514-2.479j  3.47 -0.988j -7.347-0.541j
   2.587+5.144j]]


RESULTS

Status: optimal_inaccurate
New optimal value:  15.768378744216061
New num of Constraints:  9
--------------------
new P* = 
[[ 9.    -5.537  3.313 -0.813 -1.463 -4.553]
 [-5.537 11.805  0.376  3.443 -1.124 -6.521]
 [ 3.313  0.376 10.     1.215  3.527  0.114]
 [-0.813  3.443  1.215  8.     3.692 -0.693

In [64]:
new_prob.solve(solver=cvx.CLARABEL, verbose=True)

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 17 09:29:47 AM: Your problem has 36 variables, 44 constraints, and 40 parameters.
(CVXPY) Jun 17 09:29:47 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 17 09:29:47 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 17 09:29:47 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 17 09:29:47 AM: Compiling problem (target solver=CLARABEL).
(CVXPY) Jun 17 09:29:47 AM: Reduction chain: EvalParams -> Complex2Real -> FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatr

15.731996118205018